Import Everything


In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers.core import Dropout, Activation
from keras.layers import Conv2D,BatchNormalization,MaxPool2D,Flatten,Dense
from sklearn.utils.class_weight import compute_class_weight
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Dense, Input, Activation, Dropout, GlobalAveragePooling2D, \
    BatchNormalization, concatenate, AveragePooling2D
from keras.optimizers import Adam
import time
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator

In [2]:
X = np.load() # path for Major training image data 
Y = np.load() # path for Major training class data 
print('shape of X',X.shape)
print('shape of Y',Y.shape)

shape of X (28893, 76, 76, 3)
shape of Y (28893,)


In [3]:
Y_new = []
count_bcc = 0
count_bkl = 0
count_mel = 0
count_nv = 0
count_other = 0
for i in Y:
  if i == 'bcc':
    Y_new.append(0)
    count_bcc = count_bcc + 1
  elif i == 'bkl':
    Y_new.append(1)
    count_bkl = count_bkl + 1
  elif i == 'mel':
    Y_new.append(2)
    count_mel = count_mel + 1
  elif i == 'nv':
    Y_new.append(3)
    count_nv = count_nv + 1
  elif i == 'other':
    Y_new.append(4)
    count_other = count_other + 1
print('bcc - ',count_bcc)
print('bkl - ',count_bkl)
print('mel - ', count_mel)
print('nv - ',count_nv)
print('other - ',count_other)
print(Y_new)
Y_new = np.array(Y_new)

bcc -  6646
bkl -  5248
mel -  6022
nv -  6705
other -  4272
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, Y_new, test_size=0.2, random_state=69,stratify = Y)
y_train = to_categorical(y_train, num_classes=5)
y_test = to_categorical(y_test, num_classes=5)

print('Train dataset shape',X_train.shape)
print('Test dataset shape',X_test.shape)

print(y_train.shape)

Train dataset shape (23114, 76, 76, 3)
Test dataset shape (5779, 76, 76, 3)
(23114, 5)


Model

In [5]:
def conv_layer(conv_x, filters):
    conv_x = BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
    conv_x = Conv2D(filters, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(conv_x)
    conv_x = Dropout(0.2)(conv_x)

    return conv_x


def dense_block(block_x, filters, growth_rate, layers_in_block):
    for i in range(layers_in_block):
        each_layer = conv_layer(block_x, growth_rate)
        block_x = concatenate([block_x, each_layer], axis=-1)
        filters += growth_rate

    return block_x, filters


def transition_block(trans_x, tran_filters):
    trans_x = BatchNormalization()(trans_x)
    trans_x = Activation('relu')(trans_x)
    trans_x = Conv2D(tran_filters, (1, 1), kernel_initializer='he_uniform', padding='same', use_bias=False)(trans_x)
    trans_x = AveragePooling2D((2, 2), strides=(2, 2))(trans_x)

    return trans_x, tran_filters


def dense_net(filters, growth_rate, classes, dense_block_size, layers_in_block):
    input_img = Input(shape=(76, 76, 3))
    x = Conv2D(24, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_img)
    #x = Conv2D(12, (3, 3), kernel_initializer='he_uniform', padding='same', use_bias=False)(input_img)

    dense_x = BatchNormalization()(x)
    dense_x = Activation('relu')(x)

    dense_x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(dense_x)
    for block in range(dense_block_size - 1):
        dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
        dense_x, filters = transition_block(dense_x, filters)

    dense_x, filters = dense_block(dense_x, filters, growth_rate, layers_in_block)
    dense_x = BatchNormalization()(dense_x)
    dense_x = Activation('relu')(dense_x)
    dense_x = GlobalAveragePooling2D()(dense_x)

    output = Dense(classes, activation='softmax')(dense_x)

    return Model(input_img, output)
print('done')

done


In [6]:
start = time.time()
dense_block_size = 3
layers_in_block = 4

growth_rate = 18
classes = 5
model = dense_net(growth_rate * 2, growth_rate, classes, dense_block_size, layers_in_block)
end = time.time()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 76, 76, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 76, 76, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 76, 76, 24)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 38, 38, 24)   0           activation[0][0]                 
______________________________________________________________________________________________

Training the model

In [7]:
start = time.time()

early_stopping_monitor = EarlyStopping(patience=100,monitor='val_accuracy')
model_checkpoint_callback = ModelCheckpoint(filepath='model1.h5',
                                            save_weights_only=False,
                                            monitor='val_accuracy',
                                            mode='auto',
                                            save_best_only=True,
                                            verbose=1)

batch_size = 32
epochs = 100
optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-3)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics=['accuracy'])

datagen = ImageDataGenerator(zoom_range = 0.2, horizontal_flip=True, shear_range=0.2)

datagen.fit(X_train)
history=model.fit(datagen.flow(X_train,y_train), epochs=epochs, batch_size=batch_size, shuffle=True, callbacks=[early_stopping_monitor,model_checkpoint_callback], validation_data=(X_test, y_test))
end = time.time()
print("////////////////////////////Time Taken////////////////////////////////////////")
print(end-start)

Epoch 1/100
723/723 [==============================] - 42s 45ms/step - loss: 1.2038 - accuracy: 0.4910 - val_loss: 1.1206 - val_accuracy: 0.5330

Epoch 00001: val_accuracy improved from -inf to 0.53296, saving model to model1.h5
Epoch 2/100
723/723 [==============================] - 27s 37ms/step - loss: 1.1043 - accuracy: 0.5324 - val_loss: 1.0676 - val_accuracy: 0.5285

Epoch 00002: val_accuracy did not improve from 0.53296
Epoch 3/100
723/723 [==============================] - 27s 37ms/step - loss: 1.0656 - accuracy: 0.5532 - val_loss: 1.0334 - val_accuracy: 0.5556

Epoch 00003: val_accuracy improved from 0.53296 to 0.55563, saving model to model1.h5
Epoch 4/100
723/723 [==============================] - 27s 37ms/step - loss: 1.0354 - accuracy: 0.5650 - val_loss: 1.0145 - val_accuracy: 0.5799

Epoch 00004: val_accuracy improved from 0.55563 to 0.57986, saving model to model1.h5
Epoch 5/100
723/723 [==============================] - 27s 37ms/step - loss: 1.0083 - accuracy: 0.5780 - v

In [8]:
#X.shape

Testing

In [9]:
#scores = model.evaluate(X_test, y_test, verbose=1)